In [ ]:
json_request={
    "parameter": {
        "data": {
            "idConfRoute": "279c9be3-ff41-4e91-8b9e-3615a7c0227c",
            "forbiddenPort": "",
            "status": "RUNNING",
            "runningTime": 0,
            "accuracy": 0,
            "method": "",
            "trial": 0,
            "routeDetail": [
                {
                    "idConfRouteDetail": "25f12fa1-83be-4ed9-8a30-2a59e992f926",
                    "idShip": "bda507a8-ab28-4f89-93c3-c09c4ee17ed3",
                    "lastLocationPort": "",
                    "originPort": "268f0392-8e92-4f27-9a11-5c2703cf62df",
                    "destinationPort": "fa422e2a-605a-4ca7-954b-8e2b08db4bff",
                    "mustVisitPort": "",
                    "minPort": 2,
                    "maxPort": 32,
                    "minComDays": 1,
                    "maxComDays": 14,
                    "regionRule": "NOT_SPECIFIED",
                    "roundTrip": 1,
                    "homeBase": "6a01e7c5-05e6-4ab7-a706-f0936f32a837"
                },
            ]
        }
    }
}

In [ ]:
import pandas as pd
import numpy as np
import time
from typing import List,Callable,Tuple
from collections import namedtuple
from functools import partial
import import_ipynb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from data_preparation import *
from general_function import *

import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta

now = datetime.now()+relativedelta(months=1)
month = now.month
year = now.year
start = str(year)+'-'+str(month)+'-01'
end = str(year+1)+'-'+str(month)+'-01'

input_request = input_df(json_request)

idConf = input_request[0]
forbidden = input_request[1]
df_input = input_request[2]

##EXECUTE GLOBAL VARIABLES

#master port, exclude forbiddenport
df_port_all = retrieve_port('001')
df_port = df_port_all[~df_port_all['id'].isin([forbidden])]
df_port = cleaned_port(df_port)

#master ship, include input ship only
df_ship_all = retrieve_ship('001')
df_ship = df_ship_all[df_ship_all['id'].isin(df_input.idShip)]
df_ship=df_ship.fillna(0)
ship_input=pd.merge(df_ship,df_input, how='inner', left_on=['id'], right_on=['idShip'])

#master port distance
df_portdistance = retrieve_portdistance()
df_new = cleansing_portdistance(df_port,df_portdistance)
if len(df_new)>len(df_portdistance):
    df_portdistance = copy.deepcopy(df_new)

#revenue forecast & projection
df_revenue = merge_hist_pax(start,end)

#config price cargo
df_pricecargoconfig = retrieve_pricecargoconfig()

#config low peak
df_lowpeak = retrieve_lowpeak()

#config basefare
df_basefare = retrieve_basefare()

#config cost rule
df_rulecost = retrieve_rulecost()

#price cargo flat
df_cargoflat = retrieve_cargoflat()

#price adjustment
df_adjustment = retrieve_adjustment()

#config price pnp
df_priceconfig = retrieve_priceconfig()


In [ ]:
df_lowrev, df_peakrev = separate_rev(df_revenue, df_lowpeak)

In [ ]:
start = time.time()
output_low = generate_route_beta_1(copy.deepcopy(ship_input), copy.deepcopy(df_port), copy.deepcopy(df_ship), copy.deepcopy(df_portdistance), 
                                   copy.deepcopy(df_adjustment),copy.deepcopy(df_basefare), copy.deepcopy(df_cargoflat), copy.deepcopy(df_lowpeak),
                                    copy.deepcopy(df_pricecargoconfig),copy.deepcopy(df_priceconfig),copy.deepcopy(df_lowrev), 
                                    copy.deepcopy(df_rulecost), season='LOW', npop=15, fitness_limit=2.8, generation_limit=40)
header_low = output_low[0]
est_low = output_low[1].fillna(0)
cost_low = output_low[2]
matrix_low = output_low[3]
end = time.time()
print(f"time spent for regular optimiser: {end-start}s")

In [ ]:
start = time.time()
output_peak = generate_route_beta_1(copy.deepcopy(ship_input), copy.deepcopy(df_port), copy.deepcopy(df_ship), copy.deepcopy(df_portdistance),
                                    copy.deepcopy(df_adjustment),copy.deepcopy(df_basefare),copy.deepcopy(df_cargoflat),copy.deepcopy(df_lowpeak),
                                    copy.deepcopy(df_pricecargoconfig),copy.deepcopy(df_priceconfig),copy.deepcopy(df_peakrev), 
                                    copy.deepcopy(df_rulecost), season='PEAK', npop=15, fitness_limit=2.8, generation_limit=40)
header_peak = output_peak[0]
est_peak = output_peak[1].fillna(0)
cost_peak = output_peak[2]
matrix_peak = output_peak[3]
end = time.time()
print(f"time spent for peak optimiser: {end-start}s")


header_all = pd.concat([header_low,header_peak], axis=0)
est_all = pd.concat([est_low,est_peak], axis=0)
cost_all = pd.concat([cost_low,cost_peak], axis=0)
matrix_all = pd.concat([matrix_low,matrix_peak], axis=0)
